# Simulations for evaluating archaic ancestry inference

In [317]:
# run in archanc root directory
import os
# proj_dir = os.getcwd()
proj_dir = "/mnt/norbert/home/jedidiah/projects/archanc"
msprime_dir = proj_dir +"/output/msprime/"
archie_src_dir = proj_dir + "/src/ArchIE/"
archie_out_dir = proj_dir + "/output/ArchIE/"

In [1]:
import sys
sys.path.insert(0, proj_dir + 'src/stdpopsim')

from stdpopsim import homo_sapiens
import msprime
import itertools
from random import *
import numpy as np
import pandas as pd
import math

## Function for simulating MNMs on tree

Randomly select a fraction of variant sites to be MNMs. The second mutation in each MNM is randomly placed from 1-100bp downstream of the first, and the genotypes of the first mutation are duplicated. 

To-do: separate functions for the MNM simulation from .snp and .geno file generation

In [331]:
def add_mnms(ts, model_label, output_dir="./", pop_label="pop1", rep_label=0, mnm_dist=100, mnm_frac=0.015):
    
    # .snp file names are underscore-delimited with model, population MNM parameters (dash-delimited with distance & fraction), and replicate label
    # .geno file names add the population label
#     prefix = model_label+"_"+pop_label+"_mnm"+str(mnm_dist)+"-"+str(mnm_frac)+"_"+str(rep_label)
    prefix = model_label+"_mnm"+str(mnm_dist)+"-"+str(mnm_frac)+"_"+str(rep_label)
    
    geno = np.zeros(200, dtype=np.int8)
#     eur_geno = np.zeros(100, dtype=np.int8)
#     afr_geno = np.zeros(100, dtype=np.int8)

    with open(output_dir + prefix + ".snp", "w") as text_file:
        for variant in list(ts.variants()):
            print("\t".join(["1:"+str(round(variant.site.position)), 
                             "1", 
                             str(variant.site.position/10e6), 
                             str(round(variant.site.position)), 
                             "A", 
                             "G"]), file=text_file)
            geno = np.vstack([geno, variant.genotypes])

            if random() < mnm_frac:
                dist = randint(1,mnm_dist)
                mnm_cand = variant.site.position+dist 

                print("\t".join(["1:"+str(round(mnm_cand)), 
                                 "1", 
                                 str(mnm_cand/10e6), 
                                 str(round(mnm_cand)), 
                                 "A", 
                                 "G"]), file=text_file)
                geno = np.vstack([geno, variant.genotypes])

    # delete empty first row used to initialize genotype matrix            
    geno = np.delete(geno, (0), axis=0)
    
    geno_afr = geno[:,:100]
    geno_eur = geno[:,101:200]
    
    geno_pop = [geno_afr, geno_eur]
    
    for i, pop in enumerate(["afr", "eur"]):
        np.savetxt(output_dir + prefix + "_" + pop + ".geno", geno_pop[i], delimiter="", fmt='%i')

In [336]:
# ts_rep = msprime.simulate(
#     # first 100 samples from AFR, next 100 from EUR
#     samples=[msprime.Sample(0, 0)]*sample_size + [msprime.Sample(1, 0)]*sample_size,
#     length=length, 
#     mutation_rate=mu, 
#     recombination_rate=rr,
#     random_seed=seed,
#     num_replicates=replicates,
#     **model.asdict())

# ts_rep2 = msprime.simulate(
#     # first 100 samples from AFR, next 100 from EUR
#     samples=[msprime.Sample(0, 0)]*sample_size + [msprime.Sample(1, 0)]*sample_size,
#     length=length, 
#     mutation_rate=mu, 
#     recombination_rate=rr,
#     random_seed=40,
#     num_replicates=replicates,
#     **model.asdict())

# ts_list = [ts_rep, ts_rep2]

# for ts_repi in ts_list:
#     print(ts_repi)

<generator object _replicate_generator at 0x7f203cbcb7d8>
<generator object _replicate_generator at 0x7f203cbcbd00>


## Define models and parameters

In [ ]:
# coalescent simulation parameters
sample_size = 100 # each
length = 50000
mu = 1.15e-8
rr = 1e-8
replicates = 1000
seed = 30

# MNM simulation parameters
mnm_dist = 100
mnm_frac = 0.015

# simulate African & European samples under basic Gutenkunst 3-population model
GutenkunstThreePop_model = homo_sapiens.GutenkunstThreePopOutOfAfrica()

GutenkunstThreePop_ts = msprime.simulate(
    # first 100 samples from AFR, next 100 from EUR
    samples=[msprime.Sample(0, 0)]*sample_size + [msprime.Sample(1, 0)]*sample_size,
    length=length, 
    mutation_rate=mu, 
    recombination_rate=rr,
    random_seed=seed,
    num_replicates=replicates,
    **GutenkunstThreePop_model.asdict())

#-------------------------------------------------------
# define other models here and add to model_dict below
#-------------------------------------------------------

# modify demographic parameters to include archaic branches
# GutenkunstThreePopArchaic_model = homo_sapiens.GutenkunstThreePopArchaic()

# GutenkunstThreePopArchaic_ts = msprime.simulate(
#     # first 100 samples from AFR, next 100 from EUR
#     samples=[msprime.Sample(0, 0)]*sample_size + [msprime.Sample(1, 0)]*sample_size,
#     length=length, 
#     mutation_rate=mu, 
#     recombination_rate=rr,
#     random_seed=seed,
#     num_replicates=replicates,
#     **GutenkunstThreePopArchaic_model.asdict())

model_dict = {"GutenkunstThreePop": GutenkunstThreePop_ts}

# model_dict = {"GutenkunstThreePop": GutenkunstThreePop_ts,
#              "GutenkunstThreePopArchaic": GutenkunstThreePopArchaic_ts}

## Run simulations

Simulates 200 samples (100 each of European and African ancestry) under each of the specified models (with 1000 replicates each) and adds MNMs.

Outputs the .snp and .geno files to `output/msprime/` then passes these as input to the `calc_stats_window_data.py` script from `ArchIE` to calculate the per-sample summary stats that act as features in the `ArchIE` inference method. These output files are then concatenated and passed as the test data to `ArchIE/train/train.R` to predict the fraction of archaic ancestry in each simulated sample.


In [337]:
for model_label, model in model_dict.items():

    for j, ts in enumerate(model):

        prefix = model_label + "_mnm" + str(mnm_dist) + "-" + str(mnm_frac) + "_"

        add_mnms(ts, 
                 output_dir=msprime_dir, 
                 model_label=model_label, 
                 pop_label=pop, 
                 mnm_dist=mnm_dist, 
                 mnm_frac=mnm_frac, 
                 rep_label=str(j))

        stats_afr_cmd = "python " + archie_src_dir + "data/calc_stats_window_data.py" + \
            " -s " + msprime_dir + prefix + str(j) + ".snp" + \
            " -i " + archie_src_dir + "simulations/out.ADMIXED.ind" + \
            " -a " + msprime_dir + prefix + str(j) + "_afr.geno" + \
            " -r " + msprime_dir + prefix + str(j) + "_eur.geno" + \
            " -c 1 -b 0 -e 50000 -w 50000 -z 50000 " + \
            " > " + archie_out_dir + prefix + str(j) + "_afr.txt"  
#         print(stats_afr_cmd)
        os.system(stats_afr_cmd)

        stats_eur_cmd = "python " + archie_src_dir + "data/calc_stats_window_data.py" + \
            " -s " + msprime_dir + prefix + str(j) + ".snp" + \
            " -i " + archie_src_dir + "simulations/out.ADMIXED.ind" + \
            " -a " + msprime_dir + prefix + str(j) + "_eur.geno" + \
            " -r " + msprime_dir + prefix + str(j) + "_afr.geno" + \
            " -c 1 -b 0 -e 50000 -w 50000 -z 50000 " + \
            " > " + archie_out_dir + prefix + str(j) + "_eur.txt"  
    #     print(stats_eur_cmd)
        os.system(stats_eur_cmd)

    cat_afr_cmd = "cat " + archie_out_dir + "*afr.txt" + " > " + archie_out_dir + "combined/" + prefix + "afr.txt" 
    # print(cat_cmd)
    os.system(cat_afr_cmd)

    cat_eur_cmd = "cat " + archie_out_dir + "*eur.txt" + " > " + archie_out_dir + "combined/" + prefix + "eur.txt" 
    # print(cat_cmd)
    os.system(cat_eur_cmd)

    clean_cmd = "rm " + archie_out_dir + "*"
    os.system(clean_cmd)


python /mnt/norbert/home/jedidiah/projects/archanc/src/ArchIE/data/calc_stats_window_data.py -s /mnt/norbert/home/jedidiah/projects/archanc/output/msprime/gutenkunst_mnm100-0.015_0.snp -i /mnt/norbert/home/jedidiah/projects/archanc/src/ArchIE/simulations/out.ADMIXED.ind -a /mnt/norbert/home/jedidiah/projects/archanc/output/msprime/gutenkunst_mnm100-0.015_0_afr.geno -r /mnt/norbert/home/jedidiah/projects/archanc/output/msprime/gutenkunst_mnm100-0.015_0_eur.geno -c 1 -b 0 -e 50000 -w 50000 -z 50000  > /mnt/norbert/home/jedidiah/projects/archanc/output/ArchIE/gutenkunst_mnm100-0.015_0_afr.txt
python /mnt/norbert/home/jedidiah/projects/archanc/src/ArchIE/data/calc_stats_window_data.py -s /mnt/norbert/home/jedidiah/projects/archanc/output/msprime/gutenkunst_mnm100-0.015_1.snp -i /mnt/norbert/home/jedidiah/projects/archanc/src/ArchIE/simulations/out.ADMIXED.ind -a /mnt/norbert/home/jedidiah/projects/archanc/output/msprime/gutenkunst_mnm100-0.015_1_afr.geno -r /mnt/norbert/home/jedidiah/proje

# Sandbox

In [ ]:
# ts_afr = msprime.simulate(
#     samples=[msprime.Sample(0, 0)]*100,
#     length=50000, mutation_rate=1.25e-8, random_seed=30, recombination_rate=1e-8,
#     **model.asdict())

# ts_eur = msprime.simulate(
#     samples=[msprime.Sample(1, 0)]*100,
#     length=50000, mutation_rate=1.25e-8, random_seed=30, recombination_rate=1e-8,
#     **model.asdict())

# tree=ts.first()
print(ts_afr.first().draw(format="unicode"))

In [205]:
# # simulate MNMs in AFR
# afr_geno = np.zeros(100)
# # A_new = np.array([])
# # for variant in list(itertools.islice(ts_eur.variants(), 200)):
# with open("afr_gutenkunst_mnm_100.snp", "w") as text_file:
#     for variant in list(ts_eur.variants()):
#         print("\t".join(["1:"+str(round(variant.site.position)), "1", str(variant.site.position/10e6), str(round(variant.site.position)), "A", "G"]), file=text_file)
#         afr_geno = np.vstack([afr_geno, variant.genotypes])
#     #     variant.genotypes, 
#         if random() < 0.015:
#             dist = randint(1,100)
#             mnm_cand = variant.site.position+dist 

#             print("\t".join(["1:"+str(round(mnm_cand))+"*", "1", str(mnm_cand/10e6), str(round(mnm_cand)), "A", "G"]), file=text_file)
#             afr_geno = np.vstack([afr_geno, variant.genotypes])
            
# # delete empty first row used to initialize genotype matrix            
# afr_geno = np.delete(afr_geno, (0), axis=0)
# np.savetxt("afr_gutenkunst_mnm_100.geno", afr_geno, delimiter="\t")
    
# simulate MNMs in EUR
eur_geno = np.zeros(100, dtype=np.int8)
# A_new = np.array([])
# for variant in list(itertools.islice(ts_eur.variants(), 200)):
# with open("eur_gutenkunst_mnm_100.snp", "w") as text_file:
for variant in list(ts_eur.variants()):
#         print("\t".join(["1:"+str(round(variant.site.position)), "1", str(variant.site.position/10e6), str(round(variant.site.position)), "A", "G"]), file=text_file)
    eur_geno = np.vstack([eur_geno, variant.genotypes])
#     variant.genotypes, 
    if random() < 0.015:
        dist = randint(1,100)
        mnm_cand = variant.site.position+dist 

#             print("\t".join(["1:"+str(round(mnm_cand))+"*", "1", str(mnm_cand/10e6), str(round(mnm_cand)), "A", "G"]), file=text_file)
        eur_geno = np.vstack([eur_geno, variant.genotypes])

# delete empty first row used to initialize genotype matrix
eur_geno = np.delete(eur_geno, (0), axis=0)
np.savetxt("eur_gutenkunst_mnm_100.geno", eur_geno, delimiter="\t")

In [204]:
eur_geno

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)

In [296]:
# ts_rep = msprime.simulate(
# #         samples=[msprime.Sample(i, 0)]*100,
#     samples=[msprime.Sample(gp, 0) for gp in range(2)]*100,
#     length=50000, 
#     mutation_rate=1.25e-8, 
#     recombination_rate=1e-8,
#     random_seed=30,
#     num_replicates=5,
#     **model.asdict())

# # np.repeat([msprime.Sample(gp, 0) for gp in range(2)], 100)

In [297]:
# for ts in ts_rep:
#     print(ts.genotype_matrix().shape)
# #     for node in ts.nodes():
# # #         print(1)
# # #         print(node.flags())
# #         if node.population==1:
# #             print(node)

(171, 200)
(205, 200)
(174, 200)
(211, 200)
(196, 200)


In [253]:
for node in ts_eur.nodes():
    print(node)

{'id': 0, 'time': 0.0, 'population': 1, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 1, 'time': 0.0, 'population': 1, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 2, 'time': 0.0, 'population': 1, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 3, 'time': 0.0, 'population': 1, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 4, 'time': 0.0, 'population': 1, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 5, 'time': 0.0, 'population': 1, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 6, 'time': 0.0, 'population': 1, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 7, 'time': 0.0, 'population': 1, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 8, 'time': 0.0, 'population': 1, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 9, 'time': 0.0, 'population': 1, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 10, 'time': 0.0, 'population': 1, 'individual': -1, 'metadata': b'', 'flags': 1}
{'id': 11, 'time': 0.0, 'population': 1, '

In [179]:
ts_afr_rep = msprime.simulate(
    samples=[msprime.Sample(0, 0)]*100,
    length=50000, 
    mutation_rate=1.25e-8, 
    recombination_rate=1e-8,
    random_seed=30,
    num_replicates=5,
    **model.asdict())

for j, ts in enumerate(ts_afr_rep):
    add_mnms(ts, "gutenkunst_rep"+str(j), "afr", 100, 0.015)

    
# ts_eur_rep = msprime.simulate(
#     samples=[msprime.Sample(1, 0)]*100,
#     length=50000, 
#     mutation_rate=1.25e-8, 
#     recombination_rate=1e-8,
#     random_seed=30,
#     num_replicates=5,
#     **model.asdict())

## tweaking code from ArchIE

In [214]:
ref_geno = []
with open("/mnt/norbert/home/jedidiah/projects/primeval/gutenkunst_eur_mnm100-0.015_0.geno", 'r') as f:
        for line in f:
            ll = list(line)[:-1]
            li = [int(i) for i in ll]
            ref_geno.append(li)


In [230]:
len(ref_geno[1])

100

In [215]:
geno = []
with open("/mnt/norbert/home/jedidiah/projects/primeval/gutenkunst_afr_mnm100-0.015_0.geno", 'r') as f:
        for line in f:
            ll = list(line)[:-1]
            li = [int(i) for i in ll]
            geno.append(li)

In [218]:
import sklearn.metrics.pairwise
f = np.array(geno)[:,1]
t_r = np.transpose(np.array(ref_geno))
d = []
for r in t_r:
    r_np = np.array(r)
    arr = np.array([f, r_np])
    d.append(np.max(sklearn.metrics.pairwise.pairwise_distances(arr)))


ValueError: setting an array element with a sequence.

In [221]:
arr

array([array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0]),
       array([0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])],
      dtype=object)

## Playing with recombination breakpoints

In [89]:
print(list(ts.breakpoints())[1:10])

[184.00372941605835, 270.17435065241864, 321.9186329101023, 1116.9072243191551, 2812.822350024437, 2874.662168993303, 3072.167747437993, 3506.662324887389, 3690.4728048691695]


In [92]:
from bisect import bisect_left

def takeClosest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    return after
#     if after - myNumber < myNumber - before:
#        return after
#     else:
#        return before

In [94]:
takeClosest(list(ts.breakpoints()), 2813)

2874.662168993303